##### Copyright 2019 The TensorFlow Authors.ss


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guarde y cargue un modelo usando una estrategia de distribución

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Descargar cuaderno</a></td>
</table>

## Visión general

Es común guardar y cargar un modelo durante el entrenamiento. Hay dos conjuntos de API para guardar y cargar un modelo de keras: una API de alto nivel y una API de bajo nivel. Este tutorial demuestra cómo puede usar las API de SavedModel cuando usa `tf.distribute.Strategy` . Para obtener más información sobre SavedModel y la serialización en general, lea la [guía del modelo guardado](../../guide/saved_model.ipynb) y la [guía de serialización del modelo Keras](../../guide/keras/save_and_serialize.ipynb) . Comencemos con un ejemplo simple: 

Importar dependencias:

In [ ]:
import tensorflow_datasets as tfds

import tensorflow as tf
tfds.disable_progress_bar()

Prepare los datos y el modelo usando `tf.distribute.Strategy` :

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

Entrena el modelo: 

In [ ]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

## Guarde y cargue el modelo

Ahora que tiene un modelo simple con el que trabajar, echemos un vistazo a las API de guardar / cargar. Hay dos conjuntos de API disponibles:

- High level keras `model.save` and 1`tf.keras.models.load_model`
- Low level `tf.saved_model.save` and `tf.saved_model.load` 1


### Las API de Keras

A continuación, se muestra un ejemplo de cómo guardar y cargar un modelo con las API de Keras:

In [ ]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)

Restaurar el modelo sin `tf.distribute.Strategy` :

In [ ]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

Después de restaurar el modelo, puede continuar entrenando en él, incluso sin necesidad de llamar a `compile()` nuevamente, ya que ya está compilado antes de guardarlo. El modelo se guarda en el formato proto estándar `SavedModel` TensorFlow. Para obtener más información, consulte [la guía de formato `saved_model`](../../guide/saved_model.ipynb) .

Ahora para cargar el modelo y entrenarlo usando un `tf.distribute.Strategy` :

In [ ]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Como puede ver, la carga funciona como se esperaba con `tf.distribute.Strategy` . La estrategia utilizada aquí no tiene que ser la misma estrategia utilizada antes de guardar. 

### Las API `tf.saved_model`

Ahora echemos un vistazo a las API de nivel inferior. Guardar el modelo es similar a la API de keras:

In [ ]:
model = get_model()  # get a fresh model
saved_model_path = "/tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

La carga se puede realizar con `tf.saved_model.load()` . Sin embargo, dado que es una API que se encuentra en el nivel inferior (y, por lo tanto, tiene una gama más amplia de casos de uso), no devuelve un modelo de Keras. En cambio, devuelve un objeto que contiene funciones que se pueden usar para hacer inferencias. Por ejemplo:

In [ ]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

El objeto cargado puede contener múltiples funciones, cada una asociada con una clave. El `"serving_default"` es la clave predeterminada para la función de inferencia con un modelo Keras guardado. Para hacer una inferencia con esta función: 

In [ ]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

También puede cargar y hacer inferencias de manera distribuida:

In [ ]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(
      predict_dataset)

  # Calling the function in a distributed manner
  for batch in dist_predict_dataset:
    another_strategy.run(inference_func,args=(batch,))

Llamar a la función restaurada es solo un pase hacia adelante en el modelo guardado (predecir). ¿Qué sucede si desea continuar entrenando la función cargada? ¿O incrustar la función cargada en un modelo más grande? Una práctica común es envolver este objeto cargado en una capa de Keras para lograr esto. Afortunadamente, [TF Hub](https://www.tensorflow.org/hub) tiene [hub.KerasLayer](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/keras_layer.py) para este propósito, que se muestra aquí:

In [ ]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28, 28, 1), name='input_x')
  # Wrap what's loaded to a KerasLayer
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  model.fit(train_dataset, epochs=2)

Como puede ver, `hub.KerasLayer` envuelve el resultado cargado desde `tf.saved_model.load()` en una capa de Keras que se puede usar para construir otro modelo. Esto es muy útil para el aprendizaje por transferencia. 

### ¿Qué API debo utilizar?

Para guardar, si está trabajando con un modelo de Keras, casi siempre se recomienda utilizar la API `model.save()` Keras. Si lo que está guardando no es un modelo de Keras, entonces la API de nivel inferior es su única opción.

Para la carga, la API que utilice depende de lo que desee obtener de la API de carga. Si no puede (o no desea) obtener un modelo de Keras, utilice `tf.saved_model.load()` . De lo contrario, use `tf.keras.models.load_model()` . Tenga en cuenta que puede recuperar un modelo de Keras solo si guardó un modelo de Keras.

Es posible mezclar y combinar las API. Puede guardar un modelo de Keras con `model.save` y cargar un modelo que no sea de Keras con la API de bajo nivel, `tf.saved_model.load` . 

In [ ]:
model = get_model()

# Saving the model using Keras's save() API
model.save(keras_model_path) 

another_strategy = tf.distribute.MirroredStrategy()
# Loading the model using lower level API
with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

### Advertencias

Un caso especial es cuando tiene un modelo de Keras que no tiene entradas bien definidas. Por ejemplo, un modelo secuencial se puede crear sin ninguna forma de entrada ( `Sequential([Dense(3), ...]` ). Los modelos subclasificados tampoco tienen entradas bien definidas después de la inicialización. En este caso, debe seguir con el API de nivel inferior tanto para guardar como para cargar, de lo contrario, obtendrá un error.

Para verificar si su modelo tiene entradas bien definidas, simplemente verifique si `model.inputs` es `None` . Si no es `None` , todo está bien. Las formas de entrada se definen automáticamente cuando el modelo se usa en `.fit` , `.evaluate` , `.predict` , o cuando se llama al modelo ( `model(inputs)` ).

Aquí hay un ejemplo:

In [ ]:
class SubclassedModel(tf.keras.Model):

  output_name = 'output_layer'

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
        5, dtype=tf.dtypes.float32, name=self.output_name)

  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()
# my_model.save(keras_model_path)  # ERROR! 
tf.saved_model.save(my_model, saved_model_path)